# Задача 1 Удвоение чисел и получение первого результата

In [1]:
import pathlib
data_path = pathlib.Path.cwd().joinpath('data/test_list_numbers.txt')
# Считаем содержимое файла в одну строку избавившись от переводов строк и пробелов
data_str = ""
with open(data_path, 'r') as f:
    for line in f.readlines():
        data_str += line.replace('\n','').replace(' ', '')
# убрать внешние скобки - останется только "содержимое" внешнего списка
data_str = data_str[1:-1] 
# разделить на "вложеные" списки
data_list = data_str.replace("[","").split("],")

def str_to_int_list(str_with_numbers):
    "Строку с числами преобразовать список чисел"
    l1 = str_with_numbers.split(",")
    s2i = lambda s: int(s) if s.isdigit() else None
    return list(map(s2i, l1))
# преобразовать в списки с числами
data_int = list(map(str_to_int_list,data_list))

In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def process_number(number):
    """Обработка одно числа из списка"""
    result = number * 2
    time.sleep(0.02)
    return result

def process_list(numbers):
    """Обработка списка чисео"""
    result = 0
    for number in numbers:
        if number is not None:
            result += process_number(number)
    return result

with ThreadPoolExecutor(max_workers=10) as executor:
    # Создание и запуск задач
    futures = [executor.submit(process_list, l) for l in data_int]

    # Получение результатов задач
    for future in as_completed(futures):
        first_list_sum = future.result()
        print(f"Сумма чисел в первом обработанном списке: {first_list_sum}")
        break

Сумма чисел в первом обработанном списке: 11090


# Задача 2 Поиск и суммирование чисел через цепочку файлов

In [3]:
import pathlib
import zipfile
from multiprocessing import Pool
import platform

zip_with_filenames_path = pathlib.Path.cwd().joinpath('data/path_8_8.zip')
data_zip_path = pathlib.Path.cwd().joinpath('data/recursive_challenge_8_8.zip')

In [4]:
def prepared_filename(filename):
    """Преобразование стиль пути к файлу Linux <-> Windows"""
    if platform.system()=='Linux':
        filename = filename.replace('\\','/')    
    else:
        filename = filename.replace('/', '\\')    
    return filename

def process_file(filename, zip_with_filenames_path, data_zip_path):
    """Обработать один файл"""
    # откроем архив с файлами-ссылками
    zip_with_filenames = zipfile.ZipFile(zip_with_filenames_path)
    # считаем ссылку на файл с данными
    data_file_path = prepared_filename(zip_with_filenames.read(filename).decode())
    # откроем архив с данными
    data_zip = zipfile.ZipFile(data_zip_path)
    # считаем число из data_file_path
    num = int(data_zip.read(data_file_path).decode())
    return num


In [5]:
# Получение списка файлов
paths_file = zipfile.ZipFile(zip_with_filenames_path)
files = [(p.filename, zip_with_filenames_path, data_zip_path) for p in paths_file.infolist() if not p.is_dir()]

In [6]:
%%time
# Однопоточная обработка файлов (для сравнения)
total_sum = 0
for f in files:
    total_sum += process_file(f[0], f[1], f[2])
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 3min 45s, sys: 12.6 s, total: 3min 57s
Wall time: 3min 57s


In [7]:
%%time 
# Многопроцессорная обработка файлов
total_sum = 0
with Pool(processes=12) as pool:
    for r in pool.starmap(process_file, files):
        total_sum += r
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 17.6 ms, sys: 25.8 ms, total: 43.4 ms
Wall time: 44 s
